In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
# read the dataset in from the excel file
df_t = pd.read_csv('~/CIS_4496_Project/data/raw/transactions_train.csv')
# orders = pd.read_csv('~/CIS_4496_Project/data/df_ct/df_ct.csv')
# show the first few rows
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [3]:
import polars as pl
from scripts.df_ct_combine import *
orders = combine().to_pandas()

In [4]:
orders

,customer_id,articles
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221004, 685687003, 685687004, 685687001, 5..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[688873012, 501323011, 598859003, 688873020, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 529841001, 662980002, 496111003, 5..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820043, 501820043, 674681001, 671505001, 6..."
...,...,...
1362276,fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...,"[867969008, 562245001, 867969003, 682550002, 5..."
1362277,fecc5f77b5f7ee4570efde9ab05ec94d0de2bf80efb4f6...,[915611003]
1362278,fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...,"[756322001, 822311012, 891763003, 889133001]"
1362279,fee56cc5315dafb35a4490ccc6f711092cae913550c832...,"[903647001, 903647001]"


In [5]:
# type(orders['articles'][0])

In [6]:
# from ast import literal_eval
# orders['articles'] = orders['articles'].apply(literal_eval)
# type(orders['articles'][0])

In [7]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [8]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

orders_1hot = te.fit_transform(orders['articles'])

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [9]:
# convert orders_1hot to a DataFrame
orders_1hot = pd.DataFrame(orders_1hot, columns = te.columns_)
# Inspect the first 5 rows of the DataFrame
orders_1hot.head()

,...,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,...,948152002,949198001,949551001,949551002,949594001,952267001,952938001,953450001,953763001,956217002
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [48]:
np.count_nonzero(orders_1hot['...'])

2

In [39]:
from mlxtend.frequent_patterns import fpgrowth
# the moment we have all been waiting for (again)
orders_fp = fpgrowth(orders_1hot, min_support=0.0001, max_len=3,
                use_colnames=True)

In [40]:
orders_fp

,support,itemsets
0,0.011008,(351484002)
1,0.005706,(723529001)
2,0.001078,(870304002)
3,0.000649,(750424014)
4,0.000460,(851400006)
...,...,...
73984,0.000103,"(767398003, 767438004)"
73985,0.000104,"(568570001, 568571001)"
73986,0.000120,"(534164001, 596877005)"
73987,0.000101,"(833530009, 833499010)"


In [41]:
from mlxtend.frequent_patterns import association_rules
ar = association_rules(orders_fp, 
                       metric="lift", 
                       min_threshold=0.0001)

In [42]:
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(372860002),(351484002),0.014708,0.011008,0.000352,0.023906,2.171671,0.000190,1.013214
1,(351484002),(372860002),0.011008,0.014708,0.000352,0.031942,2.171671,0.000190,1.017802
2,(351484002),(399223001),0.011008,0.011693,0.000226,0.020539,1.756522,0.000097,1.009031
3,(399223001),(351484002),0.011693,0.011008,0.000226,0.019336,1.756522,0.000097,1.008492
4,(673396002),(351484002),0.012586,0.011008,0.000344,0.027295,2.479558,0.000205,1.016744
...,...,...,...,...,...,...,...,...,...
85697,(596877005),(534164001),0.000589,0.002344,0.000120,0.204489,87.244340,0.000119,1.254107
85698,(833530009),(833499010),0.000137,0.000179,0.000101,0.736559,4112.297219,0.000101,3.795238
85699,(833499010),(833530009),0.000179,0.000137,0.000101,0.561475,4112.297219,0.000101,2.280062
85700,(559630001),(599580014),0.000379,0.003973,0.000156,0.410853,103.398641,0.000154,1.690624


In [12]:
ar.to_csv('~/CIS_4496_Project/data/ar.csv')

In [43]:
def predict(antecedent, rules, max_results= 12):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [35]:
str(df['article_id'][1])

'541518023'

In [45]:
preds = predict({str(df['article_id'][0])}, ar)
preds

/tmp/ipykernel_16850/1315413251.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return preds[:max_results]


Series([], Name: consequents, dtype: object)

In [33]:
df['article_id'][1]

541518023